In [2]:
import pandas as pd

# 1. Charger ton fichier de données (ajuste le chemin selon ton fichier)
df = pd.read_csv("base_VAR_trimestrielle.csv")  


df = df[["gdp", "cpi", "unemp", "rate_3m", "rate_10y"]]

# 3. Prendre les deux dernières lignes
histMF = df.tail(2)

# 4. Sauvegarder en CSV
histMF.to_csv("histMF.csv", index=False)


In [2]:
import pandas as pd

# Exemple fictif de données à adapter avec tes vraies valeurs
data = {
    "taux_bdt_5y": [3.25, 3.52],
    "taux_bdt_10y": [4.12, 4.38],
    "inflation_cpi": [2.3, 2.5],
    "spread_entreprise": [1.1, 1.05],
    "spread_banques": [0.9, 0.95],
    "masi": [11000, 11350],
    "masi_yield": [2.1, 2.2]
}

df = pd.DataFrame(data)
df.to_csv("histAR.csv", index=False)


In [1]:
# mapping : creation de dataframe du fonds 
import numpy as np
import pandas as pd

# Générer les dates trimestrielles de 1990 à 2025
dates = pd.date_range(start="1990-03-31", end="2025-03-31", freq="Q")

# Pour reproductibilité
np.random.seed(42)

# Générer des rendements fictifs par type de fond (valeurs en taux trimestriel)
data = {
    "Rendement_Fond_Obligataire": np.random.normal(loc=0.01, scale=0.015, size=len(dates)),   # 1% moyen
    "Rendement_Fond_Actions":     np.random.normal(loc=0.02, scale=0.05, size=len(dates)),    # 2% moyen
    "Rendement_Fond_Monetaire":   np.random.normal(loc=0.007, scale=0.003, size=len(dates)),  # 0.7% moyen
    "Rendement_Fond_Immobilier":  np.random.normal(loc=0.015, scale=0.02, size=len(dates))    # 1.5% moyen
}

# Créer le DataFrame
df_fonds = pd.DataFrame(data, index=dates)
df_fonds.index.name = "date"

# Enregistrer dans un CSV
df_fonds.to_csv("Funds_Returns_Q.csv")


C:\Users\hp\AppData\Local\Temp\ipykernel_23168\307638810.py:6: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  dates = pd.date_range(start="1990-03-31", end="2025-03-31", freq="Q")


In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# ------------------------------------------------------------------
# 1) Charger les bases
# ------------------------------------------------------------------
df_macro = pd.read_csv("base_VAR_trimestrielle.csv",
                       index_col="observation_date",
                       parse_dates=True)

df_fonds = pd.read_csv("Funds_Returns_Q.csv",
                       index_col="date",
                       parse_dates=True)

# ------------------------------------------------------------------
# 2) Créer les lags (T-1 et T-2) pour chaque variable macro
# ------------------------------------------------------------------
lags = [1, 2]
df_lagged = df_macro.copy()

for L in lags:
    tmp = df_macro.shift(L).add_suffix(f"_lag{L}")
    df_lagged = pd.concat([df_lagged, tmp], axis=1)

# Concaténer macro + fonds et supprimer les NA créés par les lags
df = pd.concat([df_lagged, df_fonds], axis=1).dropna()

# ------------------------------------------------------------------
# 3) Définir X (features) et Y (targets)
# ------------------------------------------------------------------
# Toutes les colonnes macro (avec lags) sont les features
feature_cols = df_lagged.columns
X = df[feature_cols]

# Colonnes rendements de fonds
fonds = [
    "Rendement_Fond_Obligataire",
    "Rendement_Fond_Actions",
    "Rendement_Fond_Monetaire",
    "Rendement_Fond_Immobilier"
]
Y = df[fonds]

# ------------------------------------------------------------------
# 4) Régression linéaire + extraction des coefficients et du bruit σ
# ------------------------------------------------------------------
rows = []

for fund in fonds:
    y = Y[fund]

    model = LinearRegression()              # intercept automatique
    model.fit(X, y)

    # Coefficients alignés sur feature_cols
    coef_line = list(model.coef_)           # β des features
    intercept = model.intercept_            # α

    #ruit idiosyncra Btique (écart-type des résidus)
    resid = y - model.predict(X)
    sigma = resid.std(ddof=1)

    # Concaténer : coef … + intercept + sigma
    rows.append(coef_line + [intercept, sigma])

# ------------------------------------------------------------------
# 5) Construire la matrice Mapping
# ------------------------------------------------------------------
mapping_cols = list(feature_cols) + ["intercept", "sigma"]
Mapping = pd.DataFrame(rows, index=fonds, columns=mapping_cols)

# ------------------------------------------------------------------
# 6) Sauvegarde
# ------------------------------------------------------------------
Mapping.to_csv("mapping_matrix.csv")



In [16]:
Mapping

,gdp,cpi,unemp,rate_3m,rate_10y,gdp_lag1,cpi_lag1,unemp_lag1,rate_3m_lag1,rate_10y_lag1,gdp_lag2,cpi_lag2,unemp_lag2,rate_3m_lag2,rate_10y_lag2,intercept,sigma
Rendement_Fond_Obligataire,0.000029,-0.000542,0.007074,0.004379,-0.010016,-0.000026,-0.000746,-0.006249,-0.009611,0.011307,-1.321341e-06,0.001046,-0.000212,0.006747,-0.003839,0.037632,0.013239
Rendement_Fond_Actions,0.000023,0.008766,-0.001461,-0.034183,0.011457,-0.000063,-0.010152,-0.022286,0.056335,-0.004752,1.987140e-05,0.003856,0.017516,-0.018523,-0.007717,-0.170321,0.045930
Rendement_Fond_Monetaire,0.000001,0.000088,0.000503,-0.001150,0.000370,-0.000002,-0.000222,-0.000956,0.002032,-0.000616,2.313688e-07,0.000206,0.000717,-0.000450,0.000184,-0.003104,0.002580
Rendement_Fond_Immobilier,0.000005,0.000295,-0.000800,-0.014763,0.006789,0.000002,0.000623,0.000173,0.022920,-0.005281,-6.548156e-06,-0.000887,0.000875,-0.007878,-0.000911,-0.001131,0.018817


In [18]:
Mapping.shape

(4, 17)